In [ ]:
!pip install transformers==2.6.0
!pip install seqeval
!pip install urllib3 --upgrade

Requirement already up-to-date: urllib3 in /usr/local/lib/python3.6/dist-packages (1.25.11)


In [ ]:
#Based on the Bert for NER from https://www.depends-on-the-definition.com/named-entity-recognition-with-bert/

import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm, trange

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torchtext import data
import transformers
from transformers import BertTokenizer, BertConfig, BertForTokenClassification, AdamW
from transformers import get_linear_schedule_with_warmup

from keras.preprocessing.sequence import pad_sequences
from seqeval.metrics import accuracy_score
from sklearn.metrics import f1_score, confusion_matrix

torch.manual_seed(42)
np.random.seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_path = '/content/drive/My Drive/ToxicSpan_CS669V/processed/finaltrain.pkl'
val_path = '/content/drive/My Drive/ToxicSpan_CS669V/processed/finaldev.pkl'

with open(train_path, 'rb') as f:
  train_data = pickle.load(f)
  f.close()

with open(val_path, 'rb') as f:
  val_data = pickle.load(f)
  f.close()

In [ ]:
val_data

,spans,target_final,span_final,token_final
0,"[0, 1, 2, 3, 4, 5, 6]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[(0, 7), (8, 15), (16, 21), (23, 29), (30, 37)...","[fucking, leftist, hebes, always, finding, the..."
1,"[62, 63, 64, 65]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[(0, 7), (8, 10), (11, 17), (18, 21), (22, 32)...","[because, 13, plants, are, dangerous, smdh, gr..."
2,"[28, 29, 30, 31, 32, 33, 34]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(0, 5), (6, 8), (9, 11), (12, 16), (17, 27), ...","[their, is, so, much, additional, garbage, tag..."
3,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]","[(0, 3), (4, 9), (10, 16), (17, 23), (24, 27),...","[are, there, really, enough, red, neck, idiots..."
4,"[14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 2...","[0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[(0, 4), (5, 12), (14, 15), (16, 20), (21, 26)...","[good, points, a, dumb, crude, guy, in, a, dum..."
...,...,...,...,...
789,"[58, 59, 60, 61]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[(0, 2), (3, 8), (9, 15), (16, 23), (24, 30), ...","[it, never, passes, comment, review, but, i, g..."
790,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(0, 1), (2, 5), (5, 8), (9, 14), (15, 18), (1...","[i, was, nt, there, and, support, it, 100, you..."
791,"[320, 321, 322, 323, 324, 325, 326, 327]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(0, 5), (6, 9), (10, 15), (16, 24), (25, 29),...","[funny, how, these, churches, want, to, protec..."
792,"[78, 79, 80, 81, 82, 83, 84, 85, 86]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[(0, 7), (8, 13), (14, 24), (25, 29), (30, 40)...","[typical, lying, protestor, they, exaggerate, ..."


In [ ]:
MAX_LEN = 300
BATCH_SIZE = 4

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = False)

In [ ]:
X_train = train_data['token_final']
X_val = val_data['token_final']
Y_train = train_data['target_final']
Y_val = val_data['target_final']

In [ ]:
CLASSES = {'0':0, '1':1, '[PAD]':2}

In [ ]:
def tokenize_bert(x, y):
  sentence = []
  labels = [0]
  for word, label in zip(x, y):
    tokenized_word = tokenizer.tokenize(word)
    sentence.extend(tokenized_word)
    labels.extend([label for i in range(len(tokenized_word))])
  labels.append(0)
  return(sentence, labels)

In [ ]:
  len_train = len(X_train)
  len_val = len(X_val)

  for i in range(len_train):
    X_train[i], Y_train[i] = tokenize_bert(X_train[i], Y_train[i])

  for i in range(len_val):
    X_val[i], Y_val[i] = tokenize_bert(X_val[i], Y_val[i])

In [ ]:
X_train_id = pad_sequences([tokenizer.encode(text) for text in X_train], maxlen = MAX_LEN, dtype='long', value=0.0, truncating='post', padding = 'post')
Y_train_id = pad_sequences(Y_train, maxlen=MAX_LEN, value=CLASSES['[PAD]'], dtype='long', truncating='post', padding='post')
X_val_id = pad_sequences([tokenizer.encode(text) for text in X_val], maxlen = MAX_LEN, dtype='long', value=0.0, truncating='post', padding = 'post')
Y_val_id = pad_sequences(Y_val, maxlen=MAX_LEN, value=CLASSES['[PAD]'], dtype='long', truncating='post', padding='post')

In [ ]:
def get_attention_mask(x):
  return([[(i!=0) for i in text] for text in x])

In [ ]:
attention_mask_train = get_attention_mask(X_train_id)
attention_mask_val = get_attention_mask(X_val_id)

In [ ]:
X_train_id = torch.tensor(X_train_id)
Y_train_id = torch.tensor(Y_train_id)
X_val_id = torch.tensor(X_val_id)
Y_val_id = torch.tensor(Y_val_id)
attention_mask_train = torch.tensor(attention_mask_train)
attention_mask_val = torch.tensor(attention_mask_val)

In [ ]:
train_data = TensorDataset(X_train_id, attention_mask_train, Y_train_id)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

val_data = TensorDataset(X_val_id, attention_mask_val, Y_val_id)
val_sampler = RandomSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=BATCH_SIZE)

In [ ]:
model = BertForTokenClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False
)

In [ ]:
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
FINE_TUNING = True
if FINE_TUNING:
  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'gamma', 'beta']
  optimizer_grouped_parameters = [
                                  {'params' : [p for n,p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate' : 0.01},
                                  {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],'weight_decay_rate': 0.0}]
else:
  param_optimizer = list(model.classifier.named_parameters())
  optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5, eps=1e-8)

In [ ]:
epochs = 3
max_grad_norm = 1.0

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [ ]:
train_loss, val_loss = [], []
train_acc, val_acc = [], []
train_f1, val_f1 = [], []

l = 100

for epoch in trange(epochs, desc = 'Epoch'):
  model.train()
  t_loss, t_acc = 0, 0
  predictions, true_labels = [], []
  for step, batch in enumerate(train_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_id, b_input_mask, b_labels = batch
    optimizer.zero_grad()
    outputs = model(b_input_id, token_type_ids=None, attention_mask=b_input_mask, labels = b_labels)
    loss = outputs[0]
    loss.backward()
    t_loss += loss.item()
    torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
    optimizer.step()
    scheduler.step()

    logits = outputs[1].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.extend(label_ids)
  
  print(f"Train Loss : {t_loss/len(train_dataloader)}")
  train_loss.append(t_loss/len(train_dataloader))
  pred_tags = [p_i for p, l in zip(predictions, true_labels)
                                for p_i, l_i in zip(p, l) if l_i != 2]
  valid_tags = [l_i for l in true_labels
                                for l_i in l if l_i != 2]
  train_acc.append(accuracy_score(pred_tags, valid_tags))
  train_f1.append(f1_score(pred_tags, valid_tags))
  print("Train Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
  print("Train F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
  print()

  model.eval()
  v_loss, v_accuracy = 0, 0
  predictions , true_labels = [], []
  for batch in val_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_id, b_input_mask, b_labels = batch
    with torch.no_grad():
      outputs = model(b_input_id, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    
    logits = outputs[1].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    v_loss += outputs[0].item()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.extend(label_ids)
  
  v_loss = v_loss/len(val_dataloader)
  val_loss.append(v_loss)
  if(v_loss < l):
    l = v_loss
    print("Model Checkpoint")
  torch.save(model, f'/content/drive/My Drive/model{10}.pt')
    
  print(f"Validation Loss : {v_loss}")
  pred_tags = [p_i for p, l in zip(predictions, true_labels)
                                for p_i, l_i in zip(p, l) if l_i != 2]
  valid_tags = [l_i for l in true_labels
                                for l_i in l if l_i != 2]
  print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
  print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
  val_acc.append(accuracy_score(pred_tags, valid_tags))
  val_f1.append(f1_score(pred_tags, valid_tags))
  print()

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/transformers/optimization.py:155: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


Train Loss : 0.23955748798923862
Train Accuracy: 0.9311984062382689
Train F1-Score: 0.44228857220330814

Model Checkpoint
Validation Loss : 0.2329614160900664


Epoch:  33%|███▎      | 1/3 [09:05<18:10, 545.17s/it]

Validation Accuracy: 0.9296675191815856
Validation F1-Score: 0.47091158955565743

Train Loss : 0.20191163118748037
Train Accuracy: 0.9400109273775409
Train F1-Score: 0.5470363718006287

Validation Loss : 0.25146176068383697


Epoch:  67%|██████▋   | 2/3 [18:13<09:06, 546.01s/it]

Validation Accuracy: 0.9269577396175862
Validation F1-Score: 0.5089048106448311



In [ ]:
conf = confusion_matrix(valid_tags, pred_tags)
# conf = [[31515, 170], [1797, 694]]

In [ ]:
df_cm = pd.DataFrame(conf, range(2), range(2))
# plt.figure(figsize=(10,7))
sns.set(font_scale=1.4) # for label size
sns.heatmap(df_cm, annot=True, annot_kws={"size": 12}, fmt='d') # font size

plt.show()
plt.savefig('bert.pdf')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
epoch=[]
for i in range(3):
    epoch.append(i+1)
sns.set(font_scale=1.4)

plt.figure(figsize=(7,7))
dic={'Train_Loss':train_loss,'Val_Loss': val_loss}
x=pd.DataFrame(dic)
ax = sns.lineplot(data=x)
ax.set(ylim=(0,0.8),xlabel='No. of epochs',ylabel='Loss',title='BERT')
ax.set_xticks(range(5)) # <--- s
ax.set_xticklabels(['1','2','3','4', '5'])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
epoch=[]
for i in range(3):
    epoch.append(i+1)
sns.set(font_scale=1.4)

plt.figure(figsize=(7,7))
dic={'Train_Acc':train_acc,'Val_Acc': val_acc}
x=pd.DataFrame(dic)
ax = sns.lineplot(data=x)
ax.set(ylim=(0,1),xlabel='No. of epochs',ylabel='Acc',title='BERT')
ax.set_xticks(range(5)) # <--- s
ax.set_xticklabels(['1','2','3','4', '5'])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
epoch=[]
for i in range(3):
    epoch.append(i+1)
sns.set(font_scale=1.4)

plt.figure(figsize=(7,7))
dic={'Train_F1_Score':train_f1,'Val_F1_Score': val_f1}
x=pd.DataFrame(dic)
ax = sns.lineplot(data=x)
ax.set(ylim=(0,1),xlabel='No. of epochs',ylabel='F1 Score',title='BERT')
ax.set_xticks(range(5)) # <--- s
ax.set_xticklabels(['1','2','3','4', '5'])